In [1]:
import pysdds
import logging, io

# To show logging messages in the notebook
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [2]:
# Load simple 1-page file
sdds = pysdds.read('../tests/files/sources/xLinac.matrix')

INFO:pysdds.readers.readers:Opening file "..\tests\files\sources\xLinac.matrix"
INFO:pysdds.readers.readers:Mode (auto), compression (auto), endianness (auto)
INFO:pysdds.readers.readers:Auto compression resolved as (None) from file extension
INFO:pysdds.readers.readers:Header parsed: 5 parameters, 2 arrays, 16 columns
INFO:pysdds.readers.readers:Finished in 6.299 ms
INFO:pysdds.readers.readers:Totals: 1 pages, 15 rows, 5 parameters, 2/2 arrays, 16/16 columns



In [3]:
df = sdds.parameters_to_df().T
print(df)

Page                                                   0
MinimumSingularValueRatio                           0.01
NumberOfSingularValuesUsed                            11
DeletedVectors                                          
InputFile                   /tmp/200727-101037-29553oag0
ConditionNumber                                70.407783


In [4]:
# Create a memory buffer for output - pysdds will automatically detect whether file or buffer is given
buf = io.BytesIO()

In [5]:
# Write back the loaded sdds file
pysdds.write(sdds, buf)

INFO:pysdds.writers.writers:Writing file to "<_io.BytesIO object at 0x00000212B075DF90>"
INFO:pysdds.writers.writers:Mode (ascii), compression (None), endianness (little)
INFO:pysdds.writers.writers:Header write OK
INFO:pysdds.writers.writers:Finished in 4.025 ms


In [6]:
# Inspect resulting buffer
print(buf.getvalue().decode('ascii'))

SDDS3
!Generated by pysdds 0+untagged.32.g55eaca8.dirty
!Submit issues at github.com/nikitakuklev/pysdds
&description text="Pseudo-inverse of {Transpose of 2-term fit}", &end
&parameter name=MinimumSingularValueRatio, symbol=MinimumSingularValueRatio, description="Minimum singular value ratio", type=double,  &end
&parameter name=NumberOfSingularValuesUsed, symbol=NumberOfSingularValuesUsed, description="largest singular value used", type=long,  &end
&parameter name=DeletedVectors, symbol=DeletedVectors, description="list of vectors that were deleted", type=string,  &end
&parameter name=InputFile, symbol=InputFile, description=InputFile, type=string,  &end
&parameter name=ConditionNumber, symbol=ConditionNumber, description="Condition Number", type=double,  &end
&array name=SingularValues, symbol=SingularValues, description="Singular Values", type=double,  &end
&array name=SingularValuesUsed, symbol=SingularValuesUsed, description="Singular Values Used", type=double,  &end
&column name=

In [7]:
# Explicitly request compression
buf = io.BytesIO()
pysdds.write(sdds, buf, compression='xz')

INFO:pysdds.writers.writers:Writing file to "<_io.BytesIO object at 0x00000212B09CEE00>"
INFO:pysdds.writers.writers:Mode (ascii), compression (xz), endianness (little)
INFO:pysdds.writers.writers:Header write OK
INFO:pysdds.writers.writers:Finished in 7.268 ms


In [8]:
# To change format and endianness, use sdds object methods (they will automatically update sdds data structures)
sdds.set_mode('binary')
sdds.set_endianness('big')

In [9]:
buf = io.BytesIO()
pysdds.write(sdds, buf, compression='xz')
# Note the magic bytes  0xFD, '7', 'z', 'X', 'Z', 0x00
print(buf.getvalue()[:10])
print(len(buf.getvalue()))

INFO:pysdds.writers.writers:Writing file to "<_io.BytesIO object at 0x00000212B0782CC0>"
INFO:pysdds.writers.writers:Mode (binary), compression (xz), endianness (big)
INFO:pysdds.writers.writers:Header write OK
INFO:pysdds.writers.writers:Finished in 4.892 ms


b'\xfd7zXZ\x00\x00\x04\xe6\xd6'
2600


In [10]:
buf = io.BytesIO()
pysdds.write(sdds, buf, compression='gz')
# 1f 8b is gzip marker
print(buf.getvalue()[:10])
print(len(buf.getvalue()))

INFO:pysdds.writers.writers:Writing file to "<_io.BytesIO object at 0x00000212B0782400>"
INFO:pysdds.writers.writers:Mode (binary), compression (gz), endianness (big)
INFO:pysdds.writers.writers:Header write OK
INFO:pysdds.writers.writers:Finished in 2.705 ms


b'\x1f\x8b\x08\x00\x99\xf6\xfeb\x02\xff'
2627


In [11]:
# You can of course define sdds files manually
# The best way is to first construct a dataframe, and then import it
import pandas as pd
meas_df = {'ControlName': ['foo-_~|<specialcharacterswoo>\"\'[]{}^\n\r\005', 'bar'], 'LowerLimit': [-2, -2], 'UpperLimit': [-2.0, -2.0]}
df_meas = pd.DataFrame.from_dict(meas_df)
parameters = {'par1': [1], 'par2': [1.0], 'par3': ['foo-_~|<specialcharacterswoo>\"\'[]{}^\n\r\005']}

sdds = pysdds.SDDSFile.from_df([df_meas], parameter_dict=parameters, mode='ascii')
sdds.validate_data()
buf = io.BytesIO()
pysdds.write(sdds, buf)
print(buf.getvalue().decode('ascii'))

INFO:pysdds.writers.writers:Writing file to "<_io.BytesIO object at 0x00000212B078E180>"
INFO:pysdds.writers.writers:Mode (ascii), compression (None), endianness (little)
INFO:pysdds.writers.writers:Header write OK
INFO:pysdds.writers.writers:Finished in 0.609 ms


SDDS3
!Generated by pysdds 0+untagged.32.g55eaca8.dirty
!Submit issues at github.com/nikitakuklev/pysdds
&parameter name=par1, type=long,  &end
&parameter name=par2, type=double,  &end
&parameter name=par3, type=string,  &end
&column name=ControlName, type=string,  &end
&column name=LowerLimit, type=long,  &end
&column name=UpperLimit, type=double,  &end
&data mode=ascii, &end
! page number 0
1
1.000000000000000e+00
"foo-_~|<specialcharacterswoo>\"'[]{}^\012\015\005"
2
"foo-_~|<specialcharacterswoo>\"'[]{}^\012\015\005" -2 -2.000000000000000e+00
bar -2 -2.000000000000000e+00



In [12]:
# At lowest level, data can be manually set on each object
import numpy as np
sdds = pysdds.SDDSFile()
sdds.data = pysdds.structures.Data()
sdds.set_mode('ascii')

namelist = {'name': 'col1', 'type': 'double'}
col = pysdds.structures.Column(namelist, sdds)
col.data.append(np.array([1.0,2.0]))
col.data.append(np.array([3.0,4.0]))
sdds.columns.append(col)

sdds.n_columns = 1
sdds.n_pages = 2
sdds.validate_data()
print(sdds.describe())

buf = io.BytesIO()
pysdds.write(sdds, buf)
print('-----------')
print(buf.getvalue().decode('ascii'))

INFO:pysdds.writers.writers:Writing file to "<_io.BytesIO object at 0x00000212B078E360>"
INFO:pysdds.writers.writers:Mode (ascii), compression (None), endianness (little)
INFO:pysdds.writers.writers:Header write OK
INFO:pysdds.writers.writers:Finished in 0.477 ms


SDDSFile: 2 pages, 0 parameters, 0 arrays, 1 columns, mode ascii, endianness little
 Page sizes: [2, 2]
 Column mem usage: 0.0000 MB
-----------
SDDS3
!Generated by pysdds 0+untagged.32.g55eaca8.dirty
!Submit issues at github.com/nikitakuklev/pysdds
&column name=col1, type=double,  &end
&data mode=ascii, &end
! page number 0
2
1.000000000000000e+00
2.000000000000000e+00
! page number 1
2
3.000000000000000e+00
4.000000000000000e+00

